### Load dataset from huggingface

In [88]:
from datasets import load_dataset

#dataset = load_dataset("multi_nli")
dataset = load_dataset("fever", 'v1.0')

""" 
'id', 
'label',
'claim', 
'evidence_annotation_id',
'evidence_id',
'evidence_wiki_url',
'evidence_sentence_id'
"""
train = dataset["train"]
dataset

Found cached dataset fever (/home/cas/.cache/huggingface/datasets/fever/v1.0/1.0.0/7f8936e0558704771b08c7ce9cc202071b29a0050603374507ba61d23c00a58e)
100%|██████████| 6/6 [00:00<00:00, 36.54it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 311431
    })
    labelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 37566
    })
    unlabelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 19998
    })
    unlabelled_test: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 19998
    })
    paper_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 18999
    })
    paper_test: Dataset({
        features: [

In [109]:
""" 
'id', 'text', 'lines'
"""
#wiki = load_dataset("fever", "wiki_pages")
wiki = load_dataset('data/wiki')

Using custom data configuration wiki-44edd95e7c123e85


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 361.86it/s]


DatasetGenerationError: An error occurred while generating the dataset

In [91]:
# save for later
dataset.save_to_disk("data/fever")
# load
#datasets = load_dataset('data/')

In [99]:
url = train[0]["evidence_wiki_url"]
url

'Nikolaj_Coster-Waldau'

In [107]:
wiki = wiki["wikipedia_pages"]
wiki[0]

TypeError: string indices must be integers

### Preprocess
Apply tokenization

In [56]:
from sentence_transformers import SentenceTransformer, util
import torch
# load the tranformer to create embeddings
st = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 594kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 96.8kB/s]
Downloading: 100%|██████████| 10.6k/10.6k [00:00<00:00, 4.71MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 61.1kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 191kB/s] 
Downloading: 100%|██████████| 90.9M/90.9M [00:02<00:00, 31.0MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 21.5kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 47.8kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 857kB/s] 
Downloading: 100%|██████████| 350/350 [00:00<00:00, 158kB/s]
Downloading: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.76MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 376kB/s]  
Downloading: 100%|██████████| 349/349 [00:00<00:00, 258kB/s]


In [95]:
embeds = st.encode([train[0]['premise'], train[0]['hypothesis']], convert_to_tensor=True)
# creates embeddings for both sentences in the array and returns array of size 2

#util.pytorch_cos_sim(embeds[0], embeds[1])
x = torch.cat((embeds[0], embeds[1]))
print(x.shape)

KeyError: 'premise'

In [92]:
# first ten rows for testing
sample = train[:10]

In [87]:
# map to tokenize & embed the entire dataset
def tokenize(batch):
    embeds = st.encode(
        [batch['premise'], batch['hypothesis']], 
        convert_to_tensor=True,
    )
    embeds = torch.cat((embeds[0], embeds[1]))
    return {'embed': embeds}


tk_train = train.map(
    tokenize, 
    batched=True,
)
#remove_columns=sample.column_names,

  0%|          | 0/393 [00:00<?, ?ba/s]


TypeError: string indices must be integers

In [80]:
tk_train

Dataset({
    features: ['embed'],
    num_rows: 768
})

In [31]:
# set dataset format to pytorch
tk_train.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])
tk_train.format['type']

'torch'

In [32]:
next(iter(tk_train))

{'label': tensor(1),
 'input_ids': tensor([  101, 17158,  2135,  6949,  8301, 25057,  2038,  2048,  3937,  9646,
          1011,  4031,  1998, 10505,  1012,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [11]:
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe

# load pretrained GloVe embeddings
tokenizer = get_tokenizer("basic_english")
max_words = 20
embed_len=300
global_vectors = GloVe(name='840B', dim=300)

In [123]:
import torch
# load to GPU for speed up
device = 'cuda'
# takes string, returns 6000 dim GloVe vector
def to_vector(s):
    X = tokenizer(s[0])
    # fill / cut tokens to max size
    if len(X) < max_words:
        X = X+[""]*(max_words-len(X))
    else:
        X = X[:max_words]

    X_tensor = torch.zeros(1, max_words, embed_len).to(device)
    for i, j in enumerate(X):
        X_tensor[0][i] = global_vectors.get_vecs_by_tokens(j)
    return(X_tensor.reshape(1, -1))

def combined_tensor(row):
    x = to_vector(row['premise'])
    y = to_vector(row['hypothesis'])

    #return torch.cat((x, y), 1)
    return {
        'vector': torch.cat((x, y), 1),
        'label': row['label']
    }


In [125]:
combined_tensor(
    dataset['train'][0]
)

{'vector': tensor([[-0.3423, -0.0060, -0.4845,  ...,  0.0000,  0.0000,  0.0000]],
        device='cuda:0'),
 'label': 1}

In [126]:
sample_data = load_dataset("multi_nli", split='train[:1]')

Found cached dataset multi_nli (/home/cas/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)


In [129]:
tf_datta = sample_data.with_transform(
    combined_tensor,
    columns=['premise', 'hypothesis', 'label'],
    output_all_columns=False
)

In [134]:
# the with_transform applies the custom preprocessing function on-the-fly
tf_datta.__getitem__(0)

{'vector': tensor([-0.0168, -0.2356,  0.0769,  ...,  0.0000,  0.0000,  0.0000],
        device='cuda:0'),
 'label': 1}

In [ ]:
# map tokenizer to data
tk_data = sample_data.map(
    # creates a 12k GloVe of premise + hypothesis
    combined_tensor,
    # speed up by only keeping important columns
    input_columns=['premise', 'hypothesis', 'label'],
    remove_columns=sample_data.column_names,
)

In [66]:
tk_data

Dataset({
    features: ['label', 'vector'],
    num_rows: 10
})

In [155]:
# pytorch data loading
from torch.utils.data import DataLoader

def load_dataloader(split: str, batch: int):
    ds = dataset[split].with_format('torch')
    dl = DataLoader(ds, batch_size=batch, shuffle=True)   
    return dl

train_dl = load_dataloader('train', 4)

"""
Since we have batch=4, we get 4 datapoints. 
Each index of the label tensor belongs to one text pair.
"""

# test and display dataloader for 1 batch
for idx, batch in enumerate(train_dl):
    print(batch['hypothesis'], '\n\n', batch['premise'], '\n\n', batch['label'])
    break

['The objectives were realistic.', 'I enjoyed talking with you and must go home. ', 'One of the monsters cried out in English.', 'Road camping trips will be the last thing you have left.'] 

 ['The objectives of our research were to (1) define and describe the characteristics of a worldclass finance organization, (2) identify the factors that are essential for finance organizations to improve their financial management and move towards worldclass standards, and (3) provide case studies which illustrate the efforts of leading finance organizations from private sector companies and state governments to improve their financial management and the overall performance of their organizations.', 'yeah yeah well i need to run i enjoyed talking to you', 'One of the sharp-toothed monsters cried out in a low desert language.', "and then all you'll have will be road camping trips you know let's just go out to the mountains with the car honey and uh"] 

 tensor([1, 1, 2, 0])
